In [46]:
import pandas as pd


data = pd.read_csv('Combined_data.csv', sep=';')
#On ne garde que les runs 
data = data[data['type']=='Run']
dataset = pd.DataFrame(data)  # Replace with actual table reference
dataset.drop(columns=[ 'Sr. no.', 'start_date_local', 'type','start_latlng',
       'end_latlng', 'sport_type', 'start_date', 'timezone', 'photo_count','lat', 'lng',
      'TotalPrecipitation','SnowFall', 'moving_time','MinTemperature','achievement_count' ], inplace=True)
dataset.replace({',':'.'}, regex=True, inplace=True)
dataset.head()

,distance,elapsed_time,total_elevation_gain,kudos_count,comment_count,athlete_count,average_speed,max_speed,MaxTemperature,MaxWindSpeed,SolarRadiation,SunshineDuration,RainSum
0,10879.7,4617,91.4,4,0,3,2.704,6.228,13.6,12,8.32,29249.18,0
1,1304.4,62993,0,0,0,1,1.807,4.044,3.5,17.5,4.04,10800,0
2,17503,7462,68.4,5,0,4,2.375,6.778,9.9,19.1,1.27,0,16.2
3,3457.8,2170,3.9,2,0,3,1.931,4.522,16.2,16.3,7.25,27290.56,0.1
4,10108.2,4221,6.3,5,0,1,2.449,5.462,7.9,14.3,3.54,8903.67,5.3


Faire plusieurs regressions de Lasso. 

Avec 
1) variable cible le temps de course elapsed_time
2) variable cible la distance de course
3) average speed
4) max_speed


In [47]:
X = dataset.drop(columns=['distance','elapsed_time','average_speed','max_speed'])
distance = dataset['distance']
elapsed_time = dataset['elapsed_time']
average_speed = dataset['average_speed']
max_speed = dataset['max_speed']

In [ ]:
X = X.reset_index(drop=True)
distance = distance.astype(float)
distance = distance.reset_index(drop=True)


In [ ]:
X = X.reset_index(drop=True)
distance = distance.astype(float)
distance = distance.reset_index(drop=True)


In [ ]:
model = sm.OLS(distance,X)
result = model.fit()

# Affichage du résumé des résultats
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               distance   R-squared:                       0.221
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     2.584
Date:                Sat, 22 Mar 2025   Prob (F-statistic):             0.0113
Time:                        12:49:36   Log-Likelihood:                -913.91
No. Observations:                  92   AIC:                             1848.
Df Residuals:                      82   BIC:                             1873.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 7888.2685 

In [ ]:
model = sm.OLS(distance,X)
result = model.fit()

# Affichage du résumé des résultats
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               distance   R-squared:                       0.221
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     2.584
Date:                Sat, 22 Mar 2025   Prob (F-statistic):             0.0113
Time:                        12:49:36   Log-Likelihood:                -913.91
No. Observations:                  92   AIC:                             1848.
Df Residuals:                      82   BIC:                             1873.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 7888.2685 

In [48]:
from sklearn.preprocessing import  StandardScaler
scaler_standard = StandardScaler()
X = pd.DataFrame(scaler_standard.fit_transform(X), columns=X.columns)


In [49]:
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = sm.add_constant(X)

vif_data = pd.DataFrame()
vif_data["variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print(vif_data)

               variable       VIF
0                 const  1.000000
1  total_elevation_gain  1.326048
2           kudos_count  1.385432
3         comment_count  1.071045
4         athlete_count  1.072926
5        MaxTemperature  2.660218
6          MaxWindSpeed  1.049485
7        SolarRadiation  9.513874
8      SunshineDuration  7.564583
9               RainSum  1.450637


Controler l'endogénéité, en effet le coureur comme nous l'avons remarqué sur Power Bi, peut prendre une décision de ne courir que lorsqu'il fait beau. Nous voulons contrôler cette endogénéité. 

**Modèle de Heckman**

In [50]:
X = X.reset_index(drop=True)
distance = distance.astype(float)
distance = distance.reset_index(drop=True)


In [51]:
model = sm.OLS(distance,X)
result = model.fit()

# Affichage du résumé des résultats
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:               distance   R-squared:                       0.221
Model:                            OLS   Adj. R-squared:                  0.135
Method:                 Least Squares   F-statistic:                     2.584
Date:                Sat, 22 Mar 2025   Prob (F-statistic):             0.0113
Time:                        12:49:36   Log-Likelihood:                -913.91
No. Observations:                  92   AIC:                             1848.
Df Residuals:                      82   BIC:                             1873.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 7888.2685 

In [52]:
elapsed_time = elapsed_time.astype(float)
elapsed_time = elapsed_time.reset_index(drop=True)


In [54]:
model = sm.OLS(elapsed_time,X)
result = model.fit()

# Affichage du résumé des résultats
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:           elapsed_time   R-squared:                       0.104
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.060
Date:                Sat, 22 Mar 2025   Prob (F-statistic):              0.401
Time:                        12:54:48   Log-Likelihood:                -944.00
No. Observations:                  92   AIC:                             1908.
Df Residuals:                      82   BIC:                             1933.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                 4801.8370 

In [55]:
X = X.reset_index(drop=True)
max_speed = max_speed.astype(float)
max_speed = max_speed.reset_index(drop=True)


In [ ]:
model = sm.OLS(max_speed,X)
result = model.fit()

# Affichage du résumé des résultats
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              max_speed   R-squared:                       0.114
Model:                            OLS   Adj. R-squared:                  0.017
Method:                 Least Squares   F-statistic:                     1.174
Date:                Sat, 22 Mar 2025   Prob (F-statistic):              0.323
Time:                        12:55:03   Log-Likelihood:                -210.35
No. Observations:                  92   AIC:                             440.7
Df Residuals:                      82   BIC:                             465.9
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    5.5266 

In [57]:
X = X.reset_index(drop=True)
average_speed = average_speed.astype(float)
average_speed = average_speed.reset_index(drop=True)


In [58]:
model = sm.OLS(average_speed,X)
result = model.fit()

# Affichage du résumé des résultats
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:          average_speed   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                 -0.064
Method:                 Least Squares   F-statistic:                    0.3901
Date:                Sat, 22 Mar 2025   Prob (F-statistic):              0.937
Time:                        12:56:39   Log-Likelihood:                -68.362
No. Observations:                  92   AIC:                             156.7
Df Residuals:                      82   BIC:                             181.9
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                    2.3888 